In [1]:
using Dates
using Distributed

In [11]:
function read_input(input_data)
    """
    Read in the input data to a value for N, E
    and an array of edges.
    """

    input_strings = split(input_data, "\n")
    N, E = split(input_strings[1], " ")
    N = parse(Int64, N)
    E = parse(Int64, E)
    edges = Array{Int64}(undef, 0, 2)
    new_edges = []
    # loop through and generate a complete list of edges
    for i in 2:E+1
        edge_start, edge_end = split(input_strings[i], " ")
        edge_start = parse(Int64, edge_start)
        edge_end = parse(Int64, edge_end)
        edges = vcat(edges, transpose([edge_start, edge_end]))
    end
    
    return N, E, edges
end


mutable struct Node
    """
    Node to store which node has which 
    neighbours.
    """
    node_number::Int64
    edges::Array
    domain::Array
    colour::Int64
end


function generate_nodes(N::Int64, E::Int64, edges::Array; domain_max = 2)
    """
    Generate a complete list of nodes.
    """
    nodes = Dict{Int64, Any}()
    for i in 0:(N-1)
        nodes[i] = Node(i, [], 1:domain_max, -1)
    end

    # set up the connections between the nodes
    for i in 1:E
        edge_start, edge_end = edges[i, :]
        push!(nodes[edge_start].edges, edge_end)
        push!(nodes[edge_end].edges, edge_start)
    end
    return nodes
end


function filter_neighbour_domains(nodes, node)
    """
    Whenever a colour is assigned, remove it from
    the neighbouring colour's domains.
    """
    colour = node.colour
    for node_number in node.edges
        filter!(x -> x != colour, nodes[node_number].domain)
    end
end


function check_feasibility(E, edges, nodes)
    """
    Check the feasibility of a result by 
    performing an intersect of all edges,
    and seeing if there are two values.
    """
    # first check all of the nodes have any domain left at all
    for (key, node) in nodes
        if length(node.domain) == 0
            return false
        end
    end
    # now check intersection of each node pair
    for i in 1:E
        edge_start_number, edge_end_number = edges[i, :]
        domain_set = Set(vcat(nodes[edge_start_number].domain,
                              nodes[edge_end_number].domain))
        if length(domain_set) < 2
            return false
        end
    end
    
    return true
end


function select_node(nodes; verbose = true)
    """
    Given a set of nodes, select the 'next-most-restricted'
    node to colour. This node should have the smallest domain.
    In case of a tie, select the node which has the most
    neighbours.
    """
    n_edges_max = -1
    n_domain_max = Inf
    selected_node = nothing
    for (key, node) in nodes
        n_domain = length(node.domain)
        n_edges = length(node.edges)
        # if the node has no colour yet, then check if it is 
        # the best node to use
        if node.colour == -1
            if n_domain < n_domain_max
                selected_node = node
            elseif n_domain == n_domain_max
                if n_edges > n_edges_max
                    selected_node = node
                end
            end
        end
    end
    if verbose == true
        if selected_node != nothing
            println("Selected node $(selected_node.node_number)...")
        else
            println("No nodes left for selection.")
        end
    end
    return selected_node
end 


function assign_colour(node, colour::Int64; verbose = true)
    node.colour = colour
    node.domain = [colour]
    
    if verbose == true
        println("Assigned colour $(colour) to node $(node.node_number).")
    end
end        


function initialise_search(nodes, E, edges; 
        domain_max = 2, verbose = true)
    """
    Pick the starting node as the one
    with the most neighbours, and begin 
    the search.
    """
    # at the start, we can assign colours as we wish;
    # swapping colours will just generate symmetric 
    # models.
    counter = 0
    passed = true
    while (counter <= domain_max) & (passed == true)
        # first select the starting node
        node = select_node(nodes, verbose = verbose)
        if node == nothing
            nothing
        else
            assign_colour(node, node.domain[1], verbose = verbose)
            filter_neighbour_domains(nodes, node)
            passed = check_feasibility(E, edges, nodes)
        end
        counter += 1 
    end
    
    return nodes, passed
end    


function perform_full_search(nodes, edges, E, N; verbose = true, timeout = 5)
    
    choice_cache = []
    node = 0 # dummy to ensure we enter loop
    
    t0 = Dates.now()
    t_delta = Dates.now() - t0
    max_time = Dates.Millisecond(timeout * 1000)

    while (node != nothing) & (t_delta < max_time)
        # first need to check if our current nodes have a feasible solution
        passed = check_feasibility(E, edges, nodes)
        if passed == true
            # select a node
            node = select_node(nodes, verbose = verbose)
                if node != nothing
                domain_size = length(node.domain)
                if domain_size == 1
                    assign_colour(node, node.domain[1], verbose = verbose)
                    filter_neighbour_domains(nodes, node)
                elseif domain_size > 1
                    make_choice(nodes, node, choice_cache, verbose = verbose)
                end
            end
        # if we failed the test, revert to a previous choice and re-try
        else
            nodes, node = revert_to_previous_choice(choice_cache, verbose = verbose)
        end
        t_delta = Dates.now() - t0
    end
    
    # check if we exceeded the time limit
    if t_delta < max_time
        optimality_guarantee = 1
    else
        optimality_guarantee = 0
    end
    
    # check whether our solution has any uncoloured nodes
    if nodes != nothing
        for i in 0:N-1
            if nodes[i].colour == -1
                nodes = nothing
            end
        end
    end
    
    return nodes, optimality_guarantee
end

perform_full_search (generic function with 1 method)

In [3]:
# cache the original tree once a choice is made
mutable struct Choice
    nodes::Dict # the node status at the moment the choice is made
    node # the node number we made the choice on
    choice_counter::Int64 # which choice we made
end

function make_choice(nodes, node, choice_cache::Array; verbose = true)
    """
    Whenever a choice is made, create an instance where 
    we store the status of the system at the time of the 
    choice.
    """
    # first, open up a choice instance
    node_copy = deepcopy(nodes)
    choice = Choice(node_copy, node_copy[node.node_number], 1)
    # add it to our choice cache
    push!(choice_cache, choice)
    # make the choice
    assign_colour(node, node.domain[1], verbose = false)
    filter_neighbour_domains(nodes, node)
    
    if verbose == true
        println("Made choice of colour $(node.domain[1]) for node $(node.node_number).")
    end
end


function revert_to_previous_choice(choice_cache; verbose = true)
    
    choice_made = false
    choice_nodes = nothing
    choice_node = nothing
    
    while (choice_made == false) & (length(choice_cache) >= 1)
        # increase the choice number by one
        choice = choice_cache[end]
        choice.choice_counter += 1
        available_choices = choice.node.domain
        choice_node = choice.node
        choice_nodes = choice.nodes

        if choice.choice_counter >= length(available_choices)
            choice_cache = choice_cache[1:end-1]
            choice_made = false
        else
            # print what our new choice is to screen
            if verbose == true
                println("Reverting to choose colour $(choice_node.domain[choice.choice_counter])"
                        * " for node $(choice_node.node_number)...")
            end
            assign_colour(choice_node, choice_node.domain[choice.choice_counter],
                verbose = verbose)
            filter_neighbour_domains(choice_nodes, choice_node)
            choice_made = true
        end
    end
    # return an infeasible result label if there is nowhere left
    # for us to search.
    if choice_made == false
        return nothing, nothing
    else
        return choice_nodes, choice_node
    end
end

revert_to_previous_choice (generic function with 1 method)

In [4]:
function format_result(result, N, optimal, N_colours)
    
    result_string = "$(N_colours) $(optimal) \n"
    for i in 0:N-1
        result_string *= " $(result[i].colour)"
    end
    return result_string
end

colour_graph (generic function with 1 method)

In [9]:
input_data = read("data/gc_500_9", String);
N, E, edges = read_input(input_data)

(500, 112224, [0 1; 0 2; … ; 497 499; 498 499])

In [ ]:
nodes = generate_nodes(N, E, edges, domain_max = N)
initialise_search(nodes, E, edges, domain_max = N, verbose = false)

In [14]:
nodes[288]

Node(288, Any[0, 1, 2, 3, 4, 5, 6, 7, 8, 9  …  489, 490, 491, 492, 493, 494, 495, 496, 497, 499], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12  …  491, 492, 493, 494, 495, 496, 497, 498, 499, 500], -1)